### 0. Import libraries and load data

In [51]:
%reload_ext autoreload

import os
import sys
import collections
from abc import ABC, abstractmethod

# custom scripts
sys.path.append(os.path.realpath('..'))
from utils import load_save_data

# utils
import numpy as np
from tqdm.auto import tqdm

# Hugging Face
import evaluate
from accelerate import Accelerator
from transformers import get_scheduler
from transformers import AutoTokenizer
from transformers import default_data_collator
from transformers import AutoModelForQuestionAnswering

# PyTorch 
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [52]:
data_loader = load_save_data.DataLoader()
raw_data = data_loader.load_data()
raw_data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

### 1. Preprocess the data

In [59]:
class DataPreprocessor(ABC):
    def __init__(self) -> None:
        self.max_length = 384
        self.stride = 128

    @abstractmethod
    def preprocess_train_data(self, questions_contexts):
        pass

    @abstractmethod
    def preprocess_dev_data(self, questions_contexts):
        pass

In [62]:
class DefaultDataPreprocessor(DataPreprocessor):
    def __init__(self, tokenizer, train_data_raw, dev_data_raw):
        DataPreprocessor.__init__(self)
        self.tokenizer = tokenizer
        self.train_data_raw = train_data_raw
        self.dev_data_raw = dev_data_raw

    def preprocess_train_data(self, questions_contexts):
        questions = [q.strip() for q in questions_contexts["question"]]

        preproc_questions_contexts = self.tokenizer(
            questions,
            questions_contexts["context"],
            max_length=self.max_length,
            truncation="only_second",
            stride=self.stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        offset_mapping = preproc_questions_contexts.pop("offset_mapping")
        sample_map = preproc_questions_contexts.pop("overflow_to_sample_mapping")
        answers = questions_contexts["answers"]
        start_positions = []
        end_positions = []

        for i, offset in enumerate(offset_mapping):
            sample_idx = sample_map[i]
            answer = answers[sample_idx]
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
            sequence_ids = preproc_questions_contexts.sequence_ids(i)

            idx = 0

            while sequence_ids[idx] != 1:
                idx += 1
            context_start = idx

            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            if (
                offset[context_start][0] > start_char
                or offset[context_end][1] < end_char
            ):
                start_positions.append(0)
                end_positions.append(0)
            else:
                idx = context_start

                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end

                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

        preproc_questions_contexts["start_positions"] = start_positions
        preproc_questions_contexts["end_positions"] = end_positions

        return preproc_questions_contexts

    def preprocess_dev_data(self, questions_contexts):
        questions = [q.strip() for q in questions_contexts["question"]]

        preproc_questions_contexts = self.tokenizer(
            questions,
            questions_contexts["context"],
            max_length=self.max_length,
            truncation="only_second",
            stride=self.stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        sample_map = preproc_questions_contexts.pop("overflow_to_sample_mapping")
        example_ids = []

        for i in range(len(preproc_questions_contexts["input_ids"])):
            sample_idx = sample_map[i]
            example_ids.append(questions_contexts["id"][sample_idx])

            sequence_ids = preproc_questions_contexts.sequence_ids(i)
            offset = preproc_questions_contexts["offset_mapping"][i]
            preproc_questions_contexts["offset_mapping"][i] = [
                o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
            ]

        preproc_questions_contexts["example_id"] = example_ids

        return preproc_questions_contexts

    def preprocess_data(self, dataset_name="train", verbose=True):
        if dataset_name == "train":
            preprocessed_data = self.train_data_raw.map(
                self.preprocess_train_data,
                batched=True,
                remove_columns=self.train_data_raw.column_names,
            )
            initial_pairs_no = len(self.train_data_raw)

        else:
            preprocessed_data = self.dev_data_raw.map(
                self.preprocess_dev_data,
                batched=True,
                remove_columns=self.dev_data_raw.column_names,
            )
            initial_pairs_no = len(self.dev_data_raw)

        if verbose:
            print(
                f"Nnumber of {dataset_name} question - context pairs:\nInitially: {initial_pairs_no}\nAfter preprocessing:{len(preprocessed_data)}"
            )

        return preprocessed_data

In [63]:
MODEL_CHECKPOINT = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
default_data_preprocessor = DefaultDataPreprocessor(
    tokenizer, raw_data["train"], raw_data["validation"]
)


train_dataset = default_data_preprocessor.preprocess_data("train")
validation_dataset = default_data_preprocessor.preprocess_data("validation")

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Nnumber of train question - context pairs:
Initially: 87599
After preprocessing:88524


Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Nnumber of validation question - context pairs:
Initially: 10570
After preprocessing:10784


In [ ]:
# TODO: delete this afterwards
train_dataset = train_dataset.select(range(100))
validation_dataset = validation_dataset.select(range(100))

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 100
})

### 2. Postprocess the predictions

In [ ]:
metric = evaluate.load("squad")

n_best = 20
max_answer_length = 30


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[
                            offsets[start_index][0] : offsets[end_index][1]
                        ],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [
        {"id": ex["id"], "answers": ex["answers"]} for ex in examples
    ]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

### 3. Fine-tune TinyBERT

In [11]:
train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [12]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_CHECKPOINT)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
optimizer = AdamW(model.parameters(), lr=2e-5)

In [19]:
accelerator = Accelerator()  # initially the parameter fp16=True was set
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [20]:
num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [21]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_data["validation"]
    )
    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(
        "./fine_tuned_tiny_bert_model", save_function=accelerator.save
    )

    if accelerator.is_main_process:
        tokenizer.save_pretrained("./fine_tuned_tiny_bert_tokenizer")

  0%|          | 0/13 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 0: {'exact_match': 0.02838221381267739, 'f1': 0.053914926153753935}


In [22]:
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_tiny_bert_tokenizer")
model = AutoModelForQuestionAnswering.from_pretrained("./fine_tuned_tiny_bert_model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
